# 整数制約なし最適化問題: QUIO と HUIO

通常のイジング模型やQUBOでは変数は2値のバイナリ変数 $\sigma_i \in \{-1, +1\}$ または $x_i \in \{0, 1\}$ に限定されています。しかし、実際の最適化問題では整数値の変数を扱う必要がある場合が多くあります。

OpenJijでは、変数が整数値を取ることができる最適化問題を直接解くことができます：

- **QUIO (Quadratic Unconstrained Integer Optimization)**: 2次以下の項のみを含む整数制約なし最適化問題
- **HUIO (Higher-order Unconstrained Integer Optimization)**: 高次の項も含む整数制約なし最適化問題

このチュートリアルでは、OpenJijの `sample_quio` および `sample_huio` メソッドを使って、これらの問題を解く方法を学びます。

## QUIO: 2次整数制約なし最適化問題

まず、2次以下の項のみを含む整数制約なし最適化問題（QUIO）から始めましょう。以下のようなエネルギー関数を考えます：

$$
H = c + \sum_i h_i x_i + \sum_{i<j} J_{ij} x_i x_j
$$

ここで、$x_i$ は整数変数で、各変数は指定された範囲 $[\text{lower}_i, \text{upper}_i]$ の整数値を取ることができます。

### 具体例：物流最適化問題

以下のような物流センターの配送最適化問題を考えてみましょう：

- 変数 $x_1$: 配送ルート1の配送回数（0～5回）
- 変数 $x_2$: 配送ルート2の配送回数（0～3回）
- 変数 $x_3$: 配送ルート3の配送回数（0～4回）

目的関数：
$$
H = 10 - 5x_1 - 3x_2 - 4x_3 + 2x_1x_2 + x_1x_3 + 1.5x_2x_3
$$

この問題をOpenJijで解いてみましょう。

In [ ]:
# OpenJijのインストール（必要に応じて）
!pip install openjij

In [ ]:
import openjij as oj
import numpy as np
import matplotlib.pyplot as plt

# QUIOMEPTROPOLISに対応するサンプラーを作成
sampler = oj.SASampler()

In [ ]:
# QUIO問題の定義
# 相互作用の辞書：キーはタプル、値は係数
J_quio = {
    (): 10,        # 定数項
    (1,): -5,      # x1の線形項
    (2,): -3,      # x2の線形項
    (3,): -4,      # x3の線形項
    (1, 2): 2,     # x1*x2の2次項
    (1, 3): 1,     # x1*x3の2次項
    (2, 3): 1.5    # x2*x3の2次項
}

# 変数の範囲を指定
bound_list_quio = {
    1: (0, 5),  # x1は0から5の整数
    2: (0, 3),  # x2は0から3の整数
    3: (0, 4)   # x3は0から4の整数
}

print("QUIO問題:")
print(f"相互作用: {J_quio}")
print(f"変数の範囲: {bound_list_quio}")

In [ ]:
# sample_quioメソッドでQUIO問題を解く
response_quio = sampler.sample_quio(
    J=J_quio,
    bound_list=bound_list_quio,
    num_sweeps=1000,
    num_reads=10,
    seed=12345
)

print("QUIO解法結果:")
for i, (sample, energy) in enumerate(zip(response_quio.samples(), response_quio.data_vectors['energy'])):
    print(f"解 {i+1}: {sample}, エネルギー: {energy:.3f}")
    if i >= 4:  # 最初の5つの解のみ表示
        break

In [ ]:
# 最適解の詳細分析
best_sample = response_quio.first.sample
best_energy = response_quio.first.energy

print(f"最適解: x1={best_sample[1]}, x2={best_sample[2]}, x3={best_sample[3]}")
print(f"最小エネルギー: {best_energy:.3f}")

# エネルギー値の検証
x1, x2, x3 = best_sample[1], best_sample[2], best_sample[3]
manual_energy = 10 - 5*x1 - 3*x2 - 4*x3 + 2*x1*x2 + 1*x1*x3 + 1.5*x2*x3
print(f"手動計算でのエネルギー: {manual_energy:.3f}")

## HUIO: 高次整数制約なし最適化問題

次に、高次の項も含む整数制約なし最適化問題（HUIO）を解いてみましょう。HUIOMETROPOLISは3次以上の項も扱うことができます：

$$
H = c + \sum_i h_i x_i + \sum_{i<j} J_{ij} x_i x_j + \sum_{i<j<k} K_{ijk} x_i x_j x_k + \cdots
$$

### 具体例：生産計画問題

以下のような製造業の生産計画問題を考えてみましょう：

- 変数 $y_1$: 製品Aの生産量（0～4個）
- 変数 $y_2$: 製品Bの生産量（0～3個）
- 変数 $y_3$: 製品Cの生産量（0～2個）

目的関数（製造コストの最小化）：
$$
H = 5 - 8y_1 - 6y_2 - 4y_3 + 3y_1y_2 + 2y_1y_3 + 2.5y_2y_3 + 0.5y_1y_2y_3
$$

3次項 $y_1y_2y_3$ は、3つの製品を同時に生産する際の相乗効果を表現しています。

In [ ]:
# HUIO問題の定義
J_huio = {
    (): 5,         # 定数項
    (1,): -8,      # y1の線形項
    (2,): -6,      # y2の線形項
    (3,): -4,      # y3の線形項
    (1, 2): 3,     # y1*y2の2次項
    (1, 3): 2,     # y1*y3の2次項
    (2, 3): 2.5,   # y2*y3の2次項
    (1, 2, 3): 0.5 # y1*y2*y3の3次項
}

# 変数の範囲を指定
bound_list_huio = {
    1: (0, 4),  # y1は0から4の整数
    2: (0, 3),  # y2は0から3の整数
    3: (0, 2)   # y3は0から2の整数
}

print("HUIO問題:")
print(f"相互作用: {J_huio}")
print(f"変数の範囲: {bound_list_huio}")

In [ ]:
# sample_huioメソッドでHUIO問題を解く
response_huio = sampler.sample_huio(
    J=J_huio,
    bound_list=bound_list_huio,
    num_sweeps=1000,
    num_reads=10,
    seed=12345
)

print("HUIO解法結果:")
for i, (sample, energy) in enumerate(zip(response_huio.samples(), response_huio.data_vectors['energy'])):
    print(f"解 {i+1}: {sample}, エネルギー: {energy:.3f}")
    if i >= 4:  # 最初の5つの解のみ表示
        break

In [ ]:
# 最適解の詳細分析
best_sample_huio = response_huio.first.sample
best_energy_huio = response_huio.first.energy

print(f"最適解: y1={best_sample_huio[1]}, y2={best_sample_huio[2]}, y3={best_sample_huio[3]}")
print(f"最小エネルギー: {best_energy_huio:.3f}")

# エネルギー値の検証
y1, y2, y3 = best_sample_huio[1], best_sample_huio[2], best_sample_huio[3]
manual_energy_huio = (5 - 8*y1 - 6*y2 - 4*y3 + 
                      3*y1*y2 + 2*y1*y3 + 2.5*y2*y3 + 
                      0.5*y1*y2*y3)
print(f"手動計算でのエネルギー: {manual_energy_huio:.3f}")

## 解の可視化と比較

QUIOMETROPOLISとHUIOMETROPOLISで得られた解を可視化して比較してみましょう。

In [ ]:
# エネルギー分布の可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# QUIO結果
energies_quio = response_quio.data_vectors['energy']
ax1.hist(energies_quio, bins=10, alpha=0.7, color='blue', edgecolor='black')
ax1.axvline(best_energy, color='red', linestyle='--', linewidth=2, label=f'最適解: {best_energy:.3f}')
ax1.set_xlabel('エネルギー')
ax1.set_ylabel('頻度')
ax1.set_title('QUIO エネルギー分布')
ax1.legend()
ax1.grid(True, alpha=0.3)

# HUIO結果
energies_huio = response_huio.data_vectors['energy']
ax2.hist(energies_huio, bins=10, alpha=0.7, color='green', edgecolor='black')
ax2.axvline(best_energy_huio, color='red', linestyle='--', linewidth=2, label=f'最適解: {best_energy_huio:.3f}')
ax2.set_xlabel('エネルギー')
ax2.set_ylabel('頻度')
ax2.set_title('HUIO エネルギー分布')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 解の収束性を確認
print("解の統計情報:")
print("\nQUIO:")
print(f"  最小エネルギー: {min(energies_quio):.3f}")
print(f"  平均エネルギー: {np.mean(energies_quio):.3f}")
print(f"  標準偏差: {np.std(energies_quio):.3f}")

print("\nHUIO:")
print(f"  最小エネルギー: {min(energies_huio):.3f}")
print(f"  平均エネルギー: {np.mean(energies_huio):.3f}")
print(f"  標準偏差: {np.std(energies_huio):.3f}")

## パラメータ調整の影響

最適化性能に影響を与える重要なパラメータの効果を調べてみましょう。

In [ ]:
# 異なるsweep数での性能比較
sweep_values = [100, 500, 1000, 2000]
results_comparison = []

print("sweep数による性能比較（HUIO問題）:")
for num_sweeps in sweep_values:
    response = sampler.sample_huio(
        J=J_huio,
        bound_list=bound_list_huio,
        num_sweeps=num_sweeps,
        num_reads=5,
        seed=12345
    )
    
    best_energy = response.first.energy
    mean_energy = np.mean(response.data_vectors['energy'])
    results_comparison.append((num_sweeps, best_energy, mean_energy))
    
    print(f"sweeps={num_sweeps:4d}: 最良={best_energy:7.3f}, 平均={mean_energy:7.3f}")

In [ ]:
# パラメータ比較の可視化
sweeps, best_energies, mean_energies = zip(*results_comparison)

plt.figure(figsize=(10, 6))
plt.plot(sweeps, best_energies, 'bo-', label='最良エネルギー', linewidth=2, markersize=8)
plt.plot(sweeps, mean_energies, 'rs-', label='平均エネルギー', linewidth=2, markersize=8)
plt.xlabel('Sweep数')
plt.ylabel('エネルギー')
plt.title('Sweep数による最適化性能の変化')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xscale('log')
plt.show()

## まとめ

このチュートリアルでは、OpenJijを使った整数制約なし最適化問題の解法を学びました：

### QUIO (Quadratic Unconstrained Integer Optimization)
- 2次以下の項のみを含む整数最適化問題
- `sample_quio` メソッドを使用
- 物流、スケジューリング等の応用に適している

### HUIO (Higher-order Unconstrained Integer Optimization)
- 高次の項も含む整数最適化問題
- `sample_huio` メソッドを使用
- より複雑な相互作用を表現可能
- 化学、生産計画等の高次相関がある問題に適している

### 重要なポイント
1. **変数の範囲指定**: `bound_list` で各変数の取りうる整数値の範囲を指定
2. **相互作用の表現**: 辞書形式でタプルをキーとして相互作用を定義
3. **パラメータ調整**: `num_sweeps`, `num_reads` 等のパラメータで最適化性能を調整
4. **高次項の利点**: 変数間の複雑な関係を直接表現可能

これらの手法により、バイナリ変数では表現困難な実世界の最適化問題を効率的に解くことができます。